# --------------------------------------------------------------------------------------------------------------
# PLAYAS de ESPAÑA 2017 : Riak
# --------------------------------------------------------------------------------------------------------------

## Inicializaciones previas

### a) Carga de librerías

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

# A) Carga de datos desde Excel a un DataFrame de PANDAS

Partimos de los datos normalizados (en fichero Excel). Leemos las 2 hojas del fichero Excel en un **mismo** DataFrame de Pandas para guardarlo todo junto en RIAK (desnormalizado).

In [2]:
df_excel_provincias = pd.read_excel("./data/playas_2017.xlsx", sheetname= "HOJA_PROVINCIAS")
df_excel_playas     = pd.read_excel("./data/playas_2017.xlsx", sheetname= "HOJA_PLAYAS")

In [3]:
df_riak = pd.merge(df_excel_provincias, df_excel_playas, on = ['id_provincia'], how = 'inner')
df_riak.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3511 entries, 0 to 3510
Data columns (total 12 columns):
id_provincia       3511 non-null int64
nom_provincia      3511 non-null object
comunidad          3511 non-null object
cod_playa          3511 non-null int64
nom_playa          3511 non-null object
localidad          3511 non-null object
clave_nom_playa    3511 non-null object
longitud           3511 non-null int64
longitud_txt       3511 non-null object
arena              3511 non-null object
nudista            3511 non-null object
bandera_azul       3511 non-null object
dtypes: int64(3), object(9)
memory usage: 356.6+ KB


In [4]:
df_riak.head(5)

,id_provincia,nom_provincia,comunidad,cod_playa,nom_playa,localidad,clave_nom_playa,longitud,longitud_txt,arena,nudista,bandera_azul
0,16,Las Palmas,Canarias,3001,Janubio,Puerto del Rosario,Puerto del Rosario (Janubio),230,230 metros,Dorada,Sí,No
1,16,Las Palmas,Canarias,3002,El Aljibe,La Oliva,La Oliva (El Aljibe),1210,1.210 metros,Dorada,Sí,No
2,16,Las Palmas,Canarias,3003,El Castillo,La Oliva,La Oliva (El Castillo),190,190 metros,Dorada,Sí,No
3,16,Las Palmas,Canarias,3004,Marfolín,La Oliva,La Oliva (Marfolín),950,950 metros,Dorada,Sí,No
4,16,Las Palmas,Canarias,3005,Chica,La Oliva,La Oliva (Chica),200,200 metros,Blanca,No,No


# B) BD de RIAK

## B.1) Conexión a la BD de RIAK

In [5]:
# connect to database
cliente_R = riak.RiakClient()
cliente_R.ping()

True

## B.2) Creamos los BUCKETS de RIAK

Utilizamos **2 buckets** :
   1. Uno para guardar las PLAYAS en formato JSON.
   2. Otro para guardar la LONGITUD de las PLAYAS acumulada por PROVINCIA.

En el bucket de PLAYAS utilizamos una clave autogenerada (no vamos a poder localizar las playas por clave) y en el bucket con la LONGITUD acumulada la clave es el nombre de la provincia en formato HASH, para no tener problemas ni con los espacios del nombre ni con los acentos.

Se van a crear los siguientes buckets:

|bucket|Clave|Contenido|
|-|-|-|
|playas|Marca temporal|Todos los datos del dataset en formnato JSON y desnormalizados|
|acum_longitud_provincia|Código HASH de la provincia a la que pertenece la playa|Mapa con el nombre de la provincia y la longitd de playas acumulada|

El bucket de playas tendrá el siguiente índice :

|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|
|idx_comunidad_bin|Nombre de la comunidad|Localizar playas de una comunidad concreta|

In [6]:
BUCKET_PLAYAS             = 'playas'
BUCKET_ACUM_LONGITUD_PROV = 'acum_longitud_provincia'

bucket_playas         = cliente_R.bucket(BUCKET_PLAYAS)
bucket_acum_long_prov = cliente_R.bucket_type('maps').bucket(BUCKET_ACUM_LONGITUD_PROV)

## B.3) Borramos el contenido de los BUCKETS

Definimos una **función** para eliminar los datos de un BUCKET de Riak (**en Riak no se borran los buckets**, se borran las claves de un bucket, una a una, tal y como se muestra en la siguiente función).

In [7]:
def fun_borrar_tuplas(bucket):
    for lista_de_claves in bucket.stream_keys():
        for i in lista_de_claves:
            bucket.delete(i)

In [8]:
# Borramos los datos ..
fun_borrar_tuplas(bucket_playas)
fun_borrar_tuplas(bucket_acum_long_prov)

# C) Inserción de los datos en los BUCKETS de RIAK

## C.1) Transformaciones previas

Para generar los datos en formato JSON partimos del DataFrame de Pandas y lo exportamos a JSON (en formato String) para luego cargarlo en un diccionario de Python.

** NOTA (fechas)** : cuando se genera el dato en formato JSON, los campos de tipo fecha se almacenan como un [TIMESTAMP de unix](http://www.unixtimestamp.com), por lo que habría que volver a convertirlo a fecha según el caso.

In [9]:
json_string = df_riak.to_json(orient = 'records')
json_list = json.loads(json_string)

In [10]:
print(len(json_list))

3511


In [11]:
type(json_list)

list

In [12]:
pp(json_list[0])

{
    u'arena': u'Dorada',
    u'bandera_azul': u'No',
    u'clave_nom_playa': u'Puerto del Rosario (Janubio)',
    u'cod_playa': 3001,
    u'comunidad': u'Canarias',
    u'id_provincia': 16,
    u'localidad': u'Puerto del Rosario',
    u'longitud': 230,
    u'longitud_txt': u'230 metros',
    u'nom_playa': u'Janubio',
    u'nom_provincia': u'Las Palmas',
    u'nudista': u'Sí',
}


## C.2) Inserción de los datos

Vamos a utilizar el nombre de la **provincia** como clave, por lo que necesitamos convertirla previamente a un código que no lleve ni espacios ni acentos.

Este es precisamente el objetivo de la siguiente función ...

In [13]:
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

Recorremos el dataset de **playas**, almacenando la información tanto en el bucket de **playas** como en el bucket que acumula la **longitud de playa por provincia**.

En este último utilizamos una funcionalidad que nos da Riak para guardar la información tipificada. Utilizamos un **mapa** que tiene dos elementos: 
   1. Un **registro** donde guardamos el **nombre de la provincia**
   2. Un **contador** donde guardamos la **longitud acumulada de las playas de la provincia**

In [14]:
#
# Bucle de carga de datos ...
#

for playa_json in json_list:
    key_playa     = uuid.uuid1().hex
    hash_nom_prov = hash_string(playa_json["nom_provincia"])
    hash_nom_comu = hash_string(playa_json["comunidad"])
    
    var_playa = bucket_playas.new(key_playa, playa_json)
    var_playa.add_index('idx_comunidad_bin', hash_nom_comu)   # El índice tiene que tener terminación "_bin" o "_int"
    var_playa.store()
    
    map_var_prov = bucket_acum_long_prov.new(hash_nom_prov)
    map_var_prov.registers['map_nombre_prov'].assign(playa_json["nom_provincia"].encode('utf-8'))
    map_var_prov.counters['map_metros_playa_prov'].increment(playa_json["longitud"])
    map_var_prov.store()

    

# ------------------------------------------------------------------------------
# QUERYS   (Riak)
# ------------------------------------------------------------------------------

Vamos a necesitar lo siguiente :

## 1) Una FUNCIÓN para convertir JSON --> Dataframe

En la siguiente **función** partimos del objeto JSON que nos devuelve RIAK y generamos un DataFrame Pandas.


** NOTA (fechas)** : en el caso de campos de tipo fecha, lo convertimos desde un UNIX Timestamp al formato correcto para Pandas.

In [15]:
def json_to_pandas(rows):
    # Convertimos el objeto JSON en un objeto pandas 
    df = pd.read_json(json.dumps(rows))
    
    if 'fecha' in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df = df.assign(fecha = pd.to_datetime(df.fecha, unit = 'ms'))

    return df

## 2) Un DATAFRAME con toda la info del BUCKET de PLAYAS

Cargamos todas las **playas** de la base de datos y lo guardamos en un DataFrame de Pandas, ya que hay ciertas preguntas que no pueden resolverse directamente por esta base de datos.

Recuerda que RIAK permite obtener información de una clave, pero no le es posible devolver la información **ordenada** ...

In [16]:
rows = []
for lista_de_claves in bucket_playas.stream_keys():
    for i in lista_de_claves:
        rows.append(bucket_playas.get(i).data)
        
# Convertimos el objeto json en un objeto pandas 
df_playas = json_to_pandas(rows)

In [17]:
df_playas.head()

,arena,bandera_azul,clave_nom_playa,cod_playa,comunidad,id_provincia,localidad,longitud,longitud_txt,nom_playa,nom_provincia,nudista
0,Dorada,No,Tarragona (El Milagro),1186,Cataluña/Catalunya,23,Tarragona,980,980 metros,El Milagro,Tarragona,No
1,,No,L'Ametlla de Mar (Cala Moscas),1134,Cataluña/Catalunya,23,L'Ametlla de Mar,30,30 metros,Cala Moscas,Tarragona,No
2,Dorada,No,Valdáliga (Oyambre),326,Cantabria,7,Valdáliga,1800,1.800 metros,Oyambre,Cantabria,No
3,Dorada,No,San Vicente de la Barquera (La Maza),286,Cantabria,7,San Vicente de la Barquera,150,150 metros,La Maza,Cantabria,No
4,Dorada,Sí,Noja (Trengandín),267,Cantabria,7,Noja,3700,3.700 metros,Trengandín,Cantabria,No


# ------------------------------------------------------------------------------
# QUERYS   (Riak)
# ------------------------------------------------------------------------------

### QUERY_1) Las 10 playas más largas

In [18]:
df_playas.sort_values('longitud', ascending=False)[['nom_provincia', 
                                                    'localidad', 
                                                    'nom_playa', 
                                                    'clave_nom_playa',
                                                    'longitud',
                                                    'longitud_txt']].head(10)

,nom_provincia,localidad,nom_playa,clave_nom_playa,longitud,longitud_txt
3435,Huelva,Almonte,Doñana,Almonte (Doñana),28000,28.000 metros
3224,Huelva,Almonte,Castilla,Almonte (Castilla),17000,17.000 metros
1574,Las Palmas,Pájara,El Cofete,Pájara (El Cofete),13700,13.700 metros
2125,Huelva,Lepe,Nueva Umbría,Lepe (Nueva Umbría),12000,12.000 metros
2722,Huelva,Palos de la Frontera,Mazagón,Palos de la Frontera (Mazagón),9000,9.000 metros
817,Tarragona,Deltebre,Punta del Fangar,Deltebre (Punta del Fangar),6500,6.500 metros
486,Almería,El Ejido,Punta Entinas Sabinar,El Ejido (Punta Entinas Sabinar),6500,6.500 metros
159,Girona,Sant Pere Pescador,Sant Pere Pescador,Sant Pere Pescador (Sant Pere Pescador),6400,6400 metros
3415,Tarragona,Sant Carles de la Ràpita,El Trabucador,Sant Carles de la Ràpita (El Trabucador),6165,6.165 metros
1637,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros


### QUERY_2) Las 10 playas más largas de una cierta provincia (incluído el nombre de la provincia)

In [19]:
df_playas_prov = df_playas [ df_playas["nom_provincia"] == u"A Coruña" ]   # Seleccionamos del DataFrame las 
                                                                           # playas de cierta provincia


df_playas_prov.sort_values('longitud', ascending=False)[['nom_provincia', # Orden + head
                                                         'localidad', 
                                                         'nom_playa', 
                                                         'clave_nom_playa',
                                                         'longitud',
                                                         'longitud_txt']].head(10)

,nom_provincia,localidad,nom_playa,clave_nom_playa,longitud,longitud_txt
1637,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros
537,A Coruña,Carballo,Baldaio,Carballo (Baldaio),4000,4.000 metros
712,A Coruña,Valdoviño,A Frouxeira,Valdoviño (A Frouxeira),3000,3.000 metros
430,A Coruña,Ribeira,Vilar,Ribeira (Vilar),2900,2.900 metros
923,A Coruña,Ortigueira,Morouzos,Ortigueira (Morouzos),2800,2.800 metros
28,A Coruña,Laxe,Traba,Laxe (Traba),2650,2.650 metros
2040,A Coruña,Boiro,Barraña,Boiro (Barraña),2150,2.150 metros
922,A Coruña,Porto do Son,Areas Longas,Porto do Son (Areas Longas),2100,2.100 metros
722,A Coruña,Boiro,Carragueiros,Boiro (Carragueiros),2000,2.000 metros
1636,A Coruña,Fisterra,Langosteira,Fisterra (Langosteira),1970,1.970 metros


### QUERY_3) Las playas de una comunidad autónoma concreta (ordenadas por longitud)

#### 3.1) MÉTODO_1 : Lectura directa del DataFrame

In [20]:
df_playas_comu = df_playas [ df_playas["comunidad"] == "Galicia" ]   # Seleccionamos del DataFrame las 
                                                                     # playas de cierta Comunidad Autónoa


df_playas_comu.sort_values('longitud', ascending=False)[['comunidad', # Orden + head
                                                         'nom_provincia', 
                                                         'localidad', 
                                                         'nom_playa', 
                                                         'clave_nom_playa',
                                                         'longitud',
                                                         'longitud_txt']].head(10)

,comunidad,nom_provincia,localidad,nom_playa,clave_nom_playa,longitud,longitud_txt
1637,Galicia,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros
537,Galicia,A Coruña,Carballo,Baldaio,Carballo (Baldaio),4000,4.000 metros
712,Galicia,A Coruña,Valdoviño,A Frouxeira,Valdoviño (A Frouxeira),3000,3.000 metros
430,Galicia,A Coruña,Ribeira,Vilar,Ribeira (Vilar),2900,2.900 metros
923,Galicia,A Coruña,Ortigueira,Morouzos,Ortigueira (Morouzos),2800,2.800 metros
28,Galicia,A Coruña,Laxe,Traba,Laxe (Traba),2650,2.650 metros
1062,Galicia,Pontevedra,O Grove,A Lanzada,O Grove (A Lanzada),2400,2.400 metros
1463,Galicia,Pontevedra,Redondela,Da Punta,Redondela (Da Punta),2400,2.400 metros
2040,Galicia,A Coruña,Boiro,Barraña,Boiro (Barraña),2150,2.150 metros
922,Galicia,A Coruña,Porto do Son,Areas Longas,Porto do Son (Areas Longas),2100,2.100 metros


#### 3.2) MÉTODO_2 : Lectura del DataFrame a través de un ÍNDICE

Leemos en un DataFrame (con ayuda del **ÍNDICE**) sólo las tuplas (del bucket de PLAYAS) correspondientes a la **comunidad autónoma** especificada.

In [21]:
rows = []
indices_galicia = bucket_playas.stream_index("idx_comunidad_bin", hash_string("Galicia"))
for lista_de_indices in indices_galicia.results:
    for i in lista_de_indices:
        rows.append(bucket_playas.get(i).data)
    
df_playas_Galicia = json_to_pandas(rows)

Verificamos que sólo se han seleccionado en el DataFrame las tuplas de la **comunidad autónoma** especificada (sin orden).

In [22]:
df_playas_Galicia[['comunidad',
                   'nom_provincia', 
                   'localidad', 
                   'nom_playa', 
                   'clave_nom_playa',
                   'longitud',
                   'longitud_txt']].head(10)

,comunidad,nom_provincia,localidad,nom_playa,clave_nom_playa,longitud,longitud_txt
0,Galicia,Lugo,Viveiro,Sacido,Viveiro (Sacido),130,130 metros
1,Galicia,Lugo,Foz,Llás,Foz (Llás),770,770 metros
2,Galicia,A Coruña,Ribeira,Gavoteira de Fontan,Ribeira (Gavoteira de Fontan),300,300 metros
3,Galicia,A Coruña,Ribeira,Touro,Ribeira (Touro),10,10 metros
4,Galicia,A Coruña,Ribeira,Tasca,Ribeira (Tasca),400,400 metros
5,Galicia,A Coruña,Porto do Son,Herbas Pitas,Porto do Son (Herbas Pitas),200,200 metros
6,Galicia,A Coruña,Ponteceso,Barra do Medio,Ponteceso (Barra do Medio),1500,1500 metros
7,Galicia,A Coruña,Ponteceso,O Osmo,Ponteceso (O Osmo),100,100 metros
8,Galicia,A Coruña,Ponteceso,A Barda,Ponteceso (A Barda),50,50 metros
9,Galicia,A Coruña,Oleiros,As Margaritas,Oleiros (As Margaritas),120,120 metros


Para obtener las playas de una **comunidad autónoma** en concreto si podemos utilizar el índice que habíamos creado Ad-hoc, pero para la **ordenación** necesiatmos hacerla en Pandas.

In [23]:
df_playas_Galicia.sort_values('longitud', ascending=False)[['comunidad', # Orden + head
                                                            'nom_provincia', 
                                                            'localidad', 
                                                            'nom_playa', 
                                                            'clave_nom_playa',
                                                            'longitud',
                                                            'longitud_txt']].head(10)

,comunidad,nom_provincia,localidad,nom_playa,clave_nom_playa,longitud,longitud_txt
476,Galicia,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros
31,Galicia,A Coruña,Carballo,Baldaio,Carballo (Baldaio),4000,4.000 metros
298,Galicia,A Coruña,Valdoviño,A Frouxeira,Valdoviño (A Frouxeira),3000,3.000 metros
61,Galicia,A Coruña,Ribeira,Vilar,Ribeira (Vilar),2900,2.900 metros
387,Galicia,A Coruña,Ortigueira,Morouzos,Ortigueira (Morouzos),2800,2.800 metros
141,Galicia,A Coruña,Laxe,Traba,Laxe (Traba),2650,2.650 metros
654,Galicia,Pontevedra,Redondela,Da Punta,Redondela (Da Punta),2400,2.400 metros
575,Galicia,Pontevedra,O Grove,A Lanzada,O Grove (A Lanzada),2400,2.400 metros
835,Galicia,A Coruña,Boiro,Barraña,Boiro (Barraña),2150,2.150 metros
386,Galicia,A Coruña,Porto do Son,Areas Longas,Porto do Son (Areas Longas),2100,2.100 metros


### QUERY_4) Las 10 provincias que más metros de playa tienen

La información agregada está en un **MAPA**, por lo que tenemos que procesarla y convertirla a formato JSON.

In [24]:
rows = []
for lista_de_claves in bucket_acum_long_prov.stream_keys():
    for i in lista_de_claves:
        map_var_provincias = bucket_acum_long_prov.get(i)
        rows.append({'nombre_provincia'       : map_var_provincias.registers['map_nombre_prov'].value,
                     'metros_playa_provincia' : map_var_provincias.counters['map_metros_playa_prov'].value
                    }
                   )
        
# Convertimos el objeto json en un objeto pandas 
df_map_provincias = json_to_pandas(rows)

Fácil si disponemos de un agregado. Utilizamos Pandas para ordenar la información ..,

In [25]:
df_map_provincias = df_map_provincias.sort_values('metros_playa_provincia', ascending=False).head(10)

df_map_provincias.rename(columns = {"nombre_provincia"       : "provincia",
                                    "metros_playa_provincia" : "metros_de_playa"})

,metros_de_playa,provincia
15,157040,A Coruña
13,138473,Cádiz
19,137337,Las Palmas
23,136720,Málaga
0,120765,Almería
17,118148,Alicante/Alacant
16,117767,Illes Balears
10,113720,Huelva
6,102431,Murcia
8,99015,Tarragona


### QUERY_5) Metros de playa de una cierta Comunidad Autónoma agrupados por provincia

In [26]:
df_playas_comunidad = df_playas [ df_playas["comunidad"] == "Galicia" ]   # Seleccionamos del DataFrame las 
                                                                          # playas de una cierta Comunidad Autónoma

In [27]:
df = df_playas_comunidad.groupby(["comunidad", "nom_provincia"]) [["longitud"]].sum()
df = df.sort_values('longitud', ascending = False)
df = df.rename(columns = {"longitud" : "metros_de_playa"})
df

metros_de_playa
comunidad nom_provincia                 
Galicia   A Coruña                157040
          Pontevedra               93552
          Lugo                     27217

### QUERY_6) ¿ Qué Comunidad tiene más metros de playa ?

In [28]:
df = df_playas.groupby(["comunidad"]) [["longitud"]].sum()

df = df.sort_values('longitud', ascending = False)

df.rename(columns = {"longitud" : "metros_de_playa"})

,metros_de_playa
comunidad,
Andalucía,545459
Galicia,277809
Comunitat Valenciana,276872
Cataluña/Catalunya,237274
Canarias,209374
Illes Balears,117767
"Murcia, Región de",102431
"Asturias, Principado de",70428
Cantabria,49880


### QUERY_7) ¿ Qué provincia tiene más playas ... ?

#### QUERY_7.1) ¿ Qué provincia tiene más playas ... en número ?

In [29]:
res = df_playas.groupby(["nom_provincia"]).cod_playa.nunique()
type(res)

pandas.core.series.Series

In [30]:
res.sort_values(ascending=False).head(5)

nom_provincia
A Coruña                  410
Pontevedra                370
Illes Balears             346
Las Palmas                322
Santa Cruz de Tenerife    257
Name: cod_playa, dtype: int64

In [31]:
df=res.to_frame()
((df.rename(columns={"cod_playa":"num_playas"})
  ).sort_values('num_playas', ascending = False)
).head(10)

,num_playas
nom_provincia,
A Coruña,410
Pontevedra,370
Illes Balears,346
Las Palmas,322
Santa Cruz de Tenerife,257
Asturias,205
Murcia,199
Girona,183
Alicante/Alacant,173


#### QUERY_7.2) ¿ Qué provincia tiene más playas ... con nombres diferentes ?

In [32]:
res = df_playas.groupby(["nom_provincia"]).nom_playa.nunique()
type(res)

pandas.core.series.Series

In [33]:
res.sort_values(ascending=False).head(5)

nom_provincia
A Coruña                  388
Pontevedra                350
Illes Balears             330
Las Palmas                303
Santa Cruz de Tenerife    228
Name: nom_playa, dtype: int64

In [34]:
df=res.to_frame()
((df.rename(columns={"nom_playa":"nombres_de_playas_distintos"})
  ).sort_values('nombres_de_playas_distintos', ascending = False)
).head(10)

,nombres_de_playas_distintos
nom_provincia,
A Coruña,388
Pontevedra,350
Illes Balears,330
Las Palmas,303
Santa Cruz de Tenerife,228
Asturias,199
Murcia,193
Girona,179
Alicante/Alacant,165


#### QUERY_7.3)  ¿ Qué provincia tiene más playas ... nudistas ?

In [35]:
df_playas_nudistas = df_playas [ df_playas["nudista"] == u"Sí" ]   # Seleccionamos del DataFrame las 
                                                                   # playas nudistas

In [36]:
res = df_playas_nudistas.groupby(["nom_provincia"]).size().rename("num_playas_nudistas_puras")
type(res)

pandas.core.series.Series

In [37]:
pd.DataFrame(res).sort_values('num_playas_nudistas_puras', ascending = False).head(10)

,num_playas_nudistas_puras
nom_provincia,
Las Palmas,146
Illes Balears,79
Santa Cruz de Tenerife,27
Alicante/Alacant,18
A Coruña,17
Gipuzkoa,16
Girona,16
Pontevedra,15
Tarragona,9


#### QUERY_7.4)  ¿ Qué provincia tiene más playas ... con bandera azul ?

In [38]:
df_playas_bandera_azul = df_playas [ df_playas["bandera_azul"] == u"Sí" ]   # Seleccionamos del DataFrame las 
                                                                            # playas con bandera azul

In [39]:
res = df_playas_bandera_azul.groupby(["nom_provincia"]).size().rename("num_playas_bandera_azul")
type(res)

pandas.core.series.Series

In [40]:
pd.DataFrame(res).sort_values('num_playas_bandera_azul', ascending = False).head(10)

,num_playas_bandera_azul
nom_provincia,
Alicante/Alacant,62
Pontevedra,57
A Coruña,45
Tarragona,43
Illes Balears,41
Murcia,40
Castellón/Castelló,33
Girona,31
Valencia/València,30


# FIN